# NFL Database Project: WebScraping & data collection
<br>
CIS 761 Database Management Systems – Term Project<br>
Kansas State University
<br><br>
Vishnu Bondalakunta<br>
Charles Zumbaugh<br>
James Chapman<br>
<br>

#### This notebook uses the public ESPN API. The URLs and endpoints were found in the following GitHub link. ESPN does not provide official documentation.

* ## [List of NFL API Endpoints](https://gist.github.com/nntrn/ee26cb2a0716de0947a0a4e9a157bc1c#event-competitions-api)

#### 4 tables are collected and saved as CSV files, which is used to populate the NFL database.
* games
* rosters
* athletes
* venues
* teams 

In [1]:
import pandas as pd
import numpy as np
import requests
import re

In [2]:
years = range(2013,2024) # ! Keep track of calendar year and NFL season year

ESPN_EVENTS = 'https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard?limit=1000&dates={}' #.format(year)
ESPN_ROSTERS = 'https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{}/competitions/{}/competitors/{}/roster' #.format(game_id,game_id,team_id)
ESPN_ATHLETE_INFO = 'https://site.web.api.espn.com/apis/common/v3/sports/football/nfl/athletes/{}' #.format(player_id)
ESPN_VENUE_INFO = 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/venues/{}' #.format(venue_id)
ESPN_TEAM_INFO = 'https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/{}' #.format(team_id)
ESPN_POSITION_INFO = 'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/positions/{}' #.format(position_id)

ESPN_STATISTICS = 'https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{}/competitions/{}/competitors/{}/roster/{}/statistics/0' #.format(game_id,game_id,team_id,player_id)

# Games

In [3]:
games = pd.DataFrame()
for year in years:
    try:
        events = requests.get(ESPN_EVENTS.format(year)).json()
        yearlyEvents = pd.json_normalize(events['events'])
        games = pd.concat([games, yearlyEvents], ignore_index=True)
    except: 
        print(year)

# games['competitions'] is a list of dictionaries 
games['attendance'] = games['competitions'].apply(lambda x : x[0]['attendance'])
games['venue_id'] = games['competitions'].apply(lambda x : x[0]['venue']['id'])

# games['competitions'][0]['competitors'] is a list of 2 dictionaries (home and away).
games['competitors'] = games['competitions'].apply(lambda x : x[0]['competitors'])
games['home_team_id'] = games['competitors'].apply(lambda x : x[0]['id'])
games['home_score'] = games['competitors'].apply(lambda x : x[0]['score'])
games['away_team_id'] = games['competitors'].apply(lambda x : x[1]['id'])
games['away_score'] = games['competitors'].apply(lambda x : x[1]['score'])

# games['competitors']['winner'] is a Boolean win/lose, but there are ties! 
# On 2nd thought, we could just compare the score
def tietest(x):
    try: return x[0]['winner']
    except KeyError: return False
games['home_win_bool'] = games['competitors'].apply(tietest)

def tietest(x):
    try: return x[1]['winner']
    except KeyError: return False
games['away_win_bool'] = games['competitors'].apply(tietest)

# Drop pre-season and off-season
games = games.drop(games[(games['season.type']== 1)].index)
games = games.drop(games[(games['season.type']== 4)].index)

# Drop rows
games = games[['id','season.year','season.type','week.number','date','name','shortName','attendance',
               'venue_id','home_team_id','home_score','away_team_id','away_score','home_win_bool','away_win_bool']]
games = games.rename(columns={'id':'game_id','season.year':'year','season.type':'season_type','week.number':'week'})
games.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2986 entries, 0 to 3584
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   game_id        2986 non-null   object
 1   year           2986 non-null   int64 
 2   season_type    2986 non-null   int64 
 3   week           2986 non-null   int64 
 4   date           2986 non-null   object
 5   name           2986 non-null   object
 6   shortName      2986 non-null   object
 7   attendance     2986 non-null   int64 
 8   venue_id       2986 non-null   object
 9   home_team_id   2986 non-null   object
 10  home_score     2986 non-null   object
 11  away_team_id   2986 non-null   object
 12  away_score     2986 non-null   object
 13  home_win_bool  2986 non-null   bool  
 14  away_win_bool  2986 non-null   bool  
dtypes: bool(2), int64(4), object(9)
memory usage: 332.4+ KB


# Rosters 

The rosters URL lists all players under 'entries'. But the position is something like this…<br>
$ref : "http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/positions/46?lang=en&region=us" <br>
This just splits the link at "positions" giving "/46?lang=en&region=us"
and then takes only numbers "46"




In [4]:
rosters = pd.DataFrame(columns = ["game_id", "team_id", "athlete_id","position_id","active","didNotPlay"])
def eachrow(game):
    temp_game_id = game['game_id']
    temp_home_team_id = game['home_team_id']
    temp_away_team_id = game['away_team_id']

    try:
        homeRoster = requests.get(ESPN_ROSTERS.format(temp_game_id,temp_game_id,temp_home_team_id)).json()
        for player in homeRoster['entries']:
            split = re.split("positions", player['position']['$ref'])
            position_id = re.sub("[^0-9.]", "", split[1])
            rosters.loc[len(rosters.index)] = [temp_game_id, 
                                                temp_home_team_id,
                                                player['playerId'], 
                                                position_id, 
                                                player['active'], 
                                                player['didNotPlay']]
    
        awayRoster = requests.get(ESPN_ROSTERS.format(temp_game_id,temp_game_id,temp_away_team_id)).json()
        # print(temp_game_id, temp_home_team_id, temp_away_team_id, len(homeRoster['entries']), len(awayRoster['entries']))
        for player in awayRoster['entries']:
            split = re.split("positions", player['position']['$ref'])
            position_id = re.sub("[^0-9.]", "", split[1])
            rosters.loc[len(rosters.index)] = [temp_game_id, 
                                                temp_home_team_id,
                                                player['playerId'], 
                                                position_id, 
                                                player['active'], 
                                                player['didNotPlay']]
    except:
        print(game)

games.apply(eachrow, axis=1)
# games = games.drop(columns=['competitors'])
rosters.info(verbose=True)

game_id                  330127032
year                          2012
season_type                      3
week                             4
date             2013-01-28T00:30Z
name                  Afc  at Nfc 
shortName                AFC @ NFC
attendance                       0
venue_id                      3610
home_team_id                    32
home_score                      62
away_team_id                    31
away_score                      35
home_win_bool                 True
away_win_bool                False
Name: 10, dtype: object
game_id                          340126035
year                                  2013
season_type                              3
week                                     4
date                     2014-01-27T00:30Z
name             Team Rice at Team Sanders
shortName                        RIC @ SAN
attendance                               0
venue_id                              3610
home_team_id                            35
home_score           

# Athletes 

In [8]:
athletes = pd.DataFrame()
for athlete_id in pd.unique(rosters['athlete_id']):
    try:
        athlete_info = requests.get(ESPN_ATHLETE_INFO.format(athlete_id)).json()
        athlete_info = pd.json_normalize(athlete_info['athlete'])
        athletes = pd.concat([athletes, athlete_info], ignore_index=True)
    except: 
        print(athlete_id)

# Drop rows
athletes = athletes[['id',
                     'firstName',
                     'lastName',
                     'displayName',
                     'fullName',
                     'debutYear',
                     'displayBirthPlace',
                     'displayHeight',
                     'displayWeight',
                     'displayDOB',
                     'age',
                     'displayDraft']]
                     #'college.id',
                     #'jersey',
                     #'displayJersey',
                     #'displayExperience',
                     #'position.id',
                     #'team.id',
                     #'collegeTeam.id',
                     #'collegeAthlete.id'
athletes = athletes.rename(columns={'id':'athlete_id'})

athletes.info(verbose=True)

4568981
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7166 entries, 0 to 7165
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   athlete_id         7166 non-null   object 
 1   firstName          7166 non-null   object 
 2   lastName           7166 non-null   object 
 3   displayName        7166 non-null   object 
 4   fullName           7166 non-null   object 
 5   debutYear          5871 non-null   float64
 6   displayBirthPlace  6744 non-null   object 
 7   displayHeight      7158 non-null   object 
 8   displayWeight      7158 non-null   object 
 9   displayDOB         7121 non-null   object 
 10  age                7121 non-null   float64
 11  displayDraft       4010 non-null   object 
dtypes: float64(2), object(10)
memory usage: 671.9+ KB


# Venues

In [10]:
venues = pd.DataFrame()
for venue_id in pd.unique(games['venue_id']):
    try:
        venue_info = requests.get(ESPN_VENUE_INFO.format(venue_id)).json()
        venue_info = pd.json_normalize(venue_info)
        venues = pd.concat([venues, venue_info], ignore_index=True)
    except: 
        print(venue_id)

# # Drop rows
venues = venues[['id',
                 'fullName',
                 'capacity',
                 'grass',
                 'indoor',
                 'address.city',
                 'address.state',
                 'address.zipCode']]
venues = venues.rename(columns={'id':'venue_id',
                                'address.city':'city',
                                'address.state':'state',
                                'address.zipCode':'zipCode'})
venues.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   venue_id  48 non-null     object
 1   fullName  48 non-null     object
 2   capacity  48 non-null     int64 
 3   grass     48 non-null     bool  
 4   indoor    48 non-null     bool  
 5   city      48 non-null     object
 6   state     42 non-null     object
 7   zipCode   38 non-null     object
dtypes: bool(2), int64(1), object(5)
memory usage: 2.5+ KB


# Teams

In [11]:
teams = pd.DataFrame()
for team_id in pd.unique(games['home_team_id']):
    try:
        team_info = requests.get(ESPN_TEAM_INFO.format(team_id)).json()
        team_info = pd.json_normalize(team_info)
        teams = pd.concat([teams, team_info], ignore_index=True)
    except: 
        print(team_id)

# # Drop rows
teams = teams[['team.id',
                 'team.slug',
                 'team.location',
                 'team.name',
                 'team.nickname',
                 'team.abbreviation',
                 'team.displayName',
                 'team.shortDisplayName',
                 'team.color',
                 'team.alternateColor',
                 'team.franchise.id']]
teams = teams.rename(columns={'team.id':'team_id',
                             'team.slug':'slug',
                             'team.location':'location',
                             'team.name':'name',
                             'team.nickname':'nickname',
                             'team.abbreviation':'abbreviation',
                             'team.displayName':'displayName',
                             'team.shortDisplayName':'shortDisplayName',
                             'team.color':'color',
                             'team.alternateColor':'alternateColor',
                             'team.franchise.id':'franchise_id'})

teams.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   team_id           36 non-null     object
 1   slug              36 non-null     object
 2   location          36 non-null     object
 3   name              34 non-null     object
 4   nickname          36 non-null     object
 5   abbreviation      36 non-null     object
 6   displayName       36 non-null     object
 7   shortDisplayName  36 non-null     object
 8   color             34 non-null     object
 9   alternateColor    32 non-null     object
 10  franchise_id      36 non-null     object
dtypes: object(11)
memory usage: 3.2+ KB


# Positions

In [12]:
positions = pd.DataFrame()
for position_id in pd.unique(rosters['position_id']):
    try:
        position_info = requests.get(ESPN_POSITION_INFO.format(position_id)).json()
        position_info = pd.json_normalize(position_info)
        positions = pd.concat([positions, position_info], ignore_index=True)
    except: 
        print(position_info)

# Drop rows
positions = positions[['id', 'name', 'displayName', 'abbreviation']]
positions = positions.rename(columns={'position_id':'id'})

positions.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            24 non-null     object
 1   name          24 non-null     object
 2   displayName   24 non-null     object
 3   abbreviation  24 non-null     object
dtypes: object(4)
memory usage: 896.0+ bytes


# Review

In [13]:
games.info(verbose=True)
for column in games.columns.tolist():
    try:
        print(column,'--------',pd.unique(games[column]).size)
    except:
        print(column)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2986 entries, 0 to 3584
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   game_id        2986 non-null   object
 1   year           2986 non-null   int64 
 2   season_type    2986 non-null   int64 
 3   week           2986 non-null   int64 
 4   date           2986 non-null   object
 5   name           2986 non-null   object
 6   shortName      2986 non-null   object
 7   attendance     2986 non-null   int64 
 8   venue_id       2986 non-null   object
 9   home_team_id   2986 non-null   object
 10  home_score     2986 non-null   object
 11  away_team_id   2986 non-null   object
 12  away_score     2986 non-null   object
 13  home_win_bool  2986 non-null   bool  
 14  away_win_bool  2986 non-null   bool  
dtypes: bool(2), int64(4), object(9)
memory usage: 332.4+ KB
game_id -------- 2986
year -------- 12
season_type -------- 2
week -------- 18
date -------- 1339
nam

In [14]:
rosters.info(verbose=True)
for column in rosters.columns.tolist():
    try:
        print(column,'--------',pd.unique(rosters[column]).size)
    except:
        print(column)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364111 entries, 0 to 364110
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   game_id      364111 non-null  object
 1   team_id      364111 non-null  object
 2   athlete_id   364111 non-null  int64 
 3   position_id  364111 non-null  object
 4   active       364111 non-null  bool  
 5   didNotPlay   364111 non-null  bool  
dtypes: bool(2), int64(1), object(3)
memory usage: 14.6+ MB
game_id -------- 2982
team_id -------- 36
athlete_id -------- 7167
position_id -------- 24
active -------- 2
didNotPlay -------- 2


In [22]:
pd.unique(athletes['debutYear'])

array([2003., 2004., 2005., 2006., 2008., 2009., 2010., 2011., 2012.,
       2001., 2007., 1998., 1999., 1996., 2002., 2000., 1997., 2013.,
       2014.,   nan, 2015., 2016., 2018., 1992., 2017., 1991., 2019.,
       2020., 2021.])

In [25]:
athletes['debutYear'] = athletes['debutYear'].fillna(0).astype('int64')
athletes['age'] = athletes['age'].fillna(0).astype('int64')
athletes.info(verbose=True)
for column in athletes.columns.tolist():
    try:
        print(column,'--------',pd.unique(athletes[column]).size)
    except:
        print(column)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7166 entries, 0 to 7165
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   athlete_id         7166 non-null   object
 1   firstName          7166 non-null   object
 2   lastName           7166 non-null   object
 3   displayName        7166 non-null   object
 4   fullName           7166 non-null   object
 5   debutYear          7166 non-null   int64 
 6   displayBirthPlace  6744 non-null   object
 7   displayHeight      7158 non-null   object
 8   displayWeight      7158 non-null   object
 9   displayDOB         7121 non-null   object
 10  age                7166 non-null   int64 
 11  displayDraft       4010 non-null   object
dtypes: int64(2), object(10)
memory usage: 671.9+ KB
athlete_id -------- 7166
firstName -------- 2363
lastName -------- 3862
displayName -------- 7060
fullName -------- 7060
debutYear -------- 29
displayBirthPlace -------- 2469
display

In [16]:
venues.info(verbose=True)
for column in venues.columns.tolist():
    try:
        print(column,'--------',pd.unique(venues[column]).size)
    except:
        print(column)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   venue_id  48 non-null     object
 1   fullName  48 non-null     object
 2   capacity  48 non-null     int64 
 3   grass     48 non-null     bool  
 4   indoor    48 non-null     bool  
 5   city      48 non-null     object
 6   state     42 non-null     object
 7   zipCode   38 non-null     object
dtypes: bool(2), int64(1), object(5)
memory usage: 2.5+ KB
venue_id -------- 48
fullName -------- 48
capacity -------- 2
grass -------- 2
indoor -------- 2
city -------- 43
state -------- 26
zipCode -------- 38


In [17]:
teams.info(verbose=True)
for column in teams.columns.tolist():
    try:
        print(column,'--------',pd.unique(teams[column]).size)
    except:
        print(column)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   team_id           36 non-null     object
 1   slug              36 non-null     object
 2   location          36 non-null     object
 3   name              34 non-null     object
 4   nickname          36 non-null     object
 5   abbreviation      36 non-null     object
 6   displayName       36 non-null     object
 7   shortDisplayName  36 non-null     object
 8   color             34 non-null     object
 9   alternateColor    32 non-null     object
 10  franchise_id      36 non-null     object
dtypes: object(11)
memory usage: 3.2+ KB
team_id -------- 36
slug -------- 36
location -------- 34
name -------- 35
nickname -------- 36
abbreviation -------- 36
displayName -------- 36
shortDisplayName -------- 36
color -------- 31
alternateColor -------- 22
franchise_id -------- 36


In [18]:
positions.info(verbose=True)
for column in positions.columns.tolist():
    try:
        print(column,'--------',pd.unique(positions[column]).size)
    except:
        print(column)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            24 non-null     object
 1   name          24 non-null     object
 2   displayName   24 non-null     object
 3   abbreviation  24 non-null     object
dtypes: object(4)
memory usage: 896.0+ bytes
id -------- 24
name -------- 24
displayName -------- 24
abbreviation -------- 24


# Save

In [26]:
#######################################
games.to_csv('games.csv', index=False)
rosters.to_csv('rosters.csv', index=False)
athletes.to_csv('athletes.csv', index=False)
venues.to_csv('venues.csv', index=False)
teams.to_csv('teams.csv', index=False)
positions.to_csv('positions.csv', index=False)
#######################################

In [ ]:
: )